In [2]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


In [3]:
# pip install python-snappy

In [ ]:
# source : https://forum.step.esa.int/t/extracting-pixel-values-sentinel-2a-l2a-with-snap-or-snappy/29878/3

In [7]:
path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
product_subset = ProductIO.readProduct(path_to_sentinel_data)
gc = product_subset.getSceneGeoCoding()

In [9]:
bands_names = list(product_subset.getBandNames())#[1:13]
bands_names

['Sigma0_VH_db', 'Sigma0_VH_db_count', 'Sigma0_VV_db', 'Sigma0_VV_db_count']

In [10]:
bands_names = list(product_subset.getBandNames())#[1:13]
cols = ['lat', 'lon', 'X', 'Y']
cols.extend(bands_names)
df_bands = pd.DataFrame(columns=cols)
df_bands

,lat,lon,X,Y,Sigma0_VH_db,Sigma0_VH_db_count,Sigma0_VV_db,Sigma0_VV_db_count


In [11]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
df_ksa.head()

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS
0,1,1,2.381336,96.481115,1,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC
1,1,2,2.381156,96.481115,2,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC
2,1,3,2.380977,96.481115,3,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC
3,1,4,2.380797,96.481115,4,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC
4,1,5,2.380618,96.481115,5,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC


In [ ]:
## Get for each long lat

lat, lon = (5.914702982, -73.51362747)

pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
data = list()
# print("(lat, lon) -→ (X, Y) : (%s, %s) -→ (%s, %s)" % (lat, lon, int(pixel_pos.x), int(pixel_pos.y)))

# data.append(lat)
# data.append(lon)
# data.append(int(pixel_pos.x))
# data.append(int(pixel_pos.y))

data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

for i, band_name in enumerate(bands_names):
    temp_band = product_subset.getBand(band_name)
    width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
    try:
        tmp = np.zeros(1)
        temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
        data.append(tmp[0])
    except Exception as e:
        print(band_name)
        print(width, height)
        print(int(pixel_pos.x), int(pixel_pos.y))
        print(e)
        traceback.print_exc()

df_bands.loc[df_bands.shape[0]] = data
df_bands

In [ ]:
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data

In [ ]:
bands_names = list(product_subset.getBandNames())[1:13]
cols = ['lat', 'lon', 'X', 'Y']
cols.extend(bands_names)
df_bands = pd.DataFrame(columns=cols)

In [ ]:
for i,r in tqdm(data.iterrows(), total data.shape[0]) :
    data = get_value(gc, data['lat'], data['long'])
    df_bands = df_bands.apapend(data)